**Part 3: Demo**
1.  Use the train 3D-CNN model to detect deception

# Load modules and packages

In [1]:
import cv2
from skimage.color import rgb2gray
from skimage.transform import resize
import matplotlib.pyplot as plt
import math
import os
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Conv3D, Activation, Dense, Dropout, Flatten, MaxPooling3D
import pandas as pd
# import dlib
from keras.losses import binary_crossentropy,categorical_crossentropy
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
# import tensorflow_datasets as tfds
import tensorflow as tf
from pathlib import Path
import tensorflow.keras.backend as K
import argparse

#Load 3D-CNN Model

In [3]:
model = Sequential()
model.add(Conv3D(16, kernel_size=(2,2,2), input_shape=(10,128,128,1)))
model.add(Conv3D(32, kernel_size=(2,2,2)))
model.add(MaxPooling3D(pool_size=(1,2,2)))

model.add(Conv3D(64, kernel_size=(2,2,2)))
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, kernel_size=(2,2,2)))
model.add(MaxPooling3D(pool_size=(1,2,2)))

model.add(Flatten())

model.add(Dense(5000))
model.add(Dropout(0.2))
model.add(Dense(500,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 9, 127, 127, 16)   144       
                                                                 
 conv3d_1 (Conv3D)           (None, 8, 126, 126, 32)   4128      
                                                                 
 max_pooling3d (MaxPooling3  (None, 8, 63, 63, 32)     0         
 D)                                                              
                                                                 
 conv3d_2 (Conv3D)           (None, 7, 62, 62, 64)     16448     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 3, 31, 31, 64)     0         
 g3D)                                                            
                                                                 
 conv3d_3 (Conv3D)           (None, 2, 30, 30, 64)     3

In [4]:
model.load_weights("modelWeight.h5")

# Video to Frames

In [5]:
filename = "test"

In [6]:
count = 1
success = True

print(filename)
video_path = os.path.join("%s%s" % (filename,".mp4"))
FRAMES_PATH = os.path.join("%s_%s" % (filename,"frames"))
if not os.path.isdir(FRAMES_PATH):
  os.makedirs(FRAMES_PATH)
vidcap = cv2.VideoCapture(video_path)
total_frame = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
one_frame_each = total_frame//11
print(total_frame,"/",one_frame_each)
frame_num = 0
success,image = vidcap.read()
while frame_num <= 9:
  if (count%one_frame_each == 0):                       # checks frame number and keeps one_frame_each              
    image_gray = rgb2gray(image)                                  # grayscale image
    plt.imsave("%s/%s_%d.png" % (FRAMES_PATH,filename,frame_num), image_gray, cmap= plt.cm.gray) # saves images to frame folder
    frame_num += 1
    print ('*', end="")
    success,image = vidcap.read()
  else:
    pre_image = image
    success,image = vidcap.read()                                 # reads next frame
  count += 1                                                        # loops counter

test
169 / 15
**********

# Prediction

In [7]:
frames = os.listdir(FRAMES_PATH)
frames.sort()
print(len(frames))

10


In [8]:
data = []

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

for filename in frames:

  video_path = os.path.join(FRAMES_PATH,filename)
  video = cv2.imread(video_path,cv2.IMREAD_GRAYSCALE)
  face = face_cascade.detectMultiScale(
      video,
      scaleFactor=1.08,
      minNeighbors=2,
      minSize=(48, 48)
  )

  print(filename)
  max_area=0
  area=0
  img = video

  for (x,y,w,h) in face:
      area=h*w
      if area>max_area:
          max_area=area
          img=video[y:y+h, x:x+w]
      
  img = cv2.resize(img,(128,128))
  img = preprocessing.scale(img)
  img = np.expand_dims(img, axis=2)
  img = img.astype(np.float32)
  
  data.append(img)

data = np.asarray(data).astype(np.float32)
data = np.expand_dims(data, axis=0)
print(data.shape)

test_0.png
test_1.png
test_2.png
test_3.png
test_4.png
test_5.png
test_6.png
test_7.png
test_8.png
test_9.png
(1, 10, 128, 128, 1)


In [9]:
pred = model.predict(data,batch_size=1)
if pred >= 0.5:
  print("Truth")
else:
  print("Lie")

1/1 [==============================] - 0s 105ms/step
Lie
